In [1]:
import numpy as np
import torch
import scipy
import train_func as tr

In [2]:
#数据集准备
#数据集应该存在一个字典列表里
data=np.loadtxt('irisdatamodified.data',delimiter=',')
data_part=data[:,:-1]
data_label=data[:,-1:]
dim=data_part.shape[1]
num_class=len(np.unique(data_label))
print(dim)
print('the number of class is ',num_class)
train_data1=data[0:40,:-1]
train_data2=data[50:90,:-1]
train_data3=data[100:140,:-1]
train_label1=data[0:40,-1:]
train_label2=data[50:90,-1:]
train_label3=data[100:140,-1:]
train_X=np.vstack([train_data1,train_data2,train_data3])
train_Y=np.vstack([train_label1,train_label2,train_label3])
test_data1=data[40:50,:-1]
test_data2=data[90:100,:-1]
test_data3=data[140:150,:-1]
test_label1=data[40:50,-1:]
test_label2=data[90:100,-1:]
test_label3=data[140:150,-1:]
test_X=np.vstack([test_data1,test_data2,test_data3])
test_Y=np.vstack([test_label1,test_label2,test_label3])
###########################
data_list=[]
data_dict={'weight':train_data1}
data_list.append(data_dict)
data_dict['label']=train_label1
print(data_list)
#print(data_dict)
data_dict={'weight':train_data2,'label':train_label2}
data_list.append(data_dict)
print(data_list[0]['label'])
data_dict={'weight':train_data3,'label':train_label3}
data_list.append(data_dict)
print(len(data_list))


4
the number of class is  3
[{'weight': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
   

In [8]:
def lr_train(train_sample,train_label,iter_num):
    X=train_sample
    num_samp=X.shape[0]
    unit_one=np.ones((num_samp,1))
    train_X=np.hstack((unit_one,X))
    Y=train_label
    w=np.random.random((5,1))
    print('initial w is ',w)
    loss=(1/num_samp)*np.log(np.exp(Y*np.dot(train_X,w))+1).sum()
    i=0
    while  i<iter_num:
        sum_delta=((Y*train_X)/(1+np.exp(Y*np.dot(train_X,w)))).sum(axis=0)
        w=(w.T+(1/num_samp)*sum_delta).T
        print('w is ',w)
        loss=(1/num_samp)*np.log(np.exp(Y*np.dot(train_X,w))+1).sum()
        i=i+1
        #loss>0 or
    return w

In [21]:
#开空字典，把对应类名变成字符，作为键名，值里存计算结果
#训练
weights={}
iter_numb=1000
tr.test_val()
for i_class_1 in range(num_class-1):
    for i_class_2 in range(num_class)[i_class_1+1:]:
        print(i_class_1,i_class_2)
        x=np.vstack([data_list[i_class_1]['weight'],data_list[i_class_2]['weight']])
        y=np.vstack([data_list[i_class_1]['label'],data_list[i_class_2]['label']])
        w=tr.lr_train(x,y,iter_numb)
        #print('w is ',w)
        weights[str(i_class_1)+str(i_class_2)]=w#w是返回来的权重
        
print(weights)


#test
predict_lb={}
for i_class_t1 in range(num_class-1):
    for i_class_t2 in range(num_class)[i_class_t1+1:]:
        print(i_class_t1,i_class_t2)
        predict_lb[str(i_class_t1)+str(i_class_t2)]=tr.lr_test(weights[str(i_class_t1)+str(i_class_t2)],test_X,i_class_t1,i_class_t2)
        
print(predict_lb)
vote_mat=np.zeros([test_X.shape[0],len(predict_lb)])
t=0
for X_s in predict_lb.values():
    vote_mat[:,t]=X_s[:,0]
    t=t+1
vote_mat=vote_mat+1
vote_mat=vote_mat.astype(int)#强制转换成整数 用astype
print(vote_mat)
result_mat=np.zeros([test_X.shape[0],1],dtype=int)
t=0
for i_s in range(test_X.shape[0]):
    result_mat[t,0]=np.argmax(np.bincount(vote_mat[t,:]))
    t=t+1
    
print(result_mat)
print('num of test sampe',result_mat.shape[0])

succeed
0 1
initial w is  [[0.31472207]
 [0.38668947]
 [0.64353196]
 [0.76262023]
 [0.54179543]]
0 2
initial w is  [[0.34106147]
 [0.04283714]
 [0.31563842]
 [0.87700074]
 [0.91908535]]
1 2
initial w is  [[0.16510036]
 [0.91748516]
 [0.48512524]
 [0.06877481]
 [0.58099092]]
{'01': array([[ 0.74445111],
       [ 0.49528871],
       [ 3.49022875],
       [-4.61856431],
       [-1.76713882]]), '02': array([[ 0.89190005],
       [ 0.74184967],
       [ 2.76051053],
       [-3.68497826],
       [-1.28037382]]), '12': array([[  7.73423649],
       [  9.6769656 ],
       [  8.49422452],
       [-14.82888753],
       [-12.71922811]])}
0 1
0 2
1 2
{'01': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [